In [ ]:
from google.colab import files

uploaded = files.upload()

df = pd.read_csv('car_data.csv') 3`
df.head()


Saving car_data.csv to car_data.csv


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [ ]:
df.info()
df.describe()
df.isna().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


,0
name,0
year,0
selling_price,0
km_driven,0
fuel,0
seller_type,0
transmission,0
owner,0
mileage,221
engine,221


In [ ]:
df.columns


Index(['name', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'mileage', 'engine', 'max_power', 'torque',
       'seats'],
      dtype='object')

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error
import joblib


In [ ]:
df = pd.read_csv("car_data.csv")
df.head()
df.columns


Index(['name', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'mileage', 'engine', 'max_power', 'torque',
       'seats'],
      dtype='object')

In [ ]:
# Remove non-digits and convert to numeric
df['engine'] = df['engine'].str.extract('(\d+)').astype(float)
df['max_power'] = df['max_power'].str.extract('(\d+\.?\d*)').astype(float)


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-264450942.py:2: SyntaxWarning: invalid escape sequence '\d'
  df['engine'] = df['engine'].str.extract('(\d+)').astype(float)
/tmp/ipython-input-264450942.py:3: SyntaxWarning: invalid escape sequence '\d'
  df['max_power'] = df['max_power'].str.extract('(\d+\.?\d*)').astype(float)


In [ ]:
df_model = df[['year', 'km_driven', 'fuel', 'seller_type',
               'transmission', 'owner', 'seats', 'engine', 'max_power', 'selling_price']]


In [ ]:
# Clean owner column
df['owner'] = df['owner'].replace({
    'First Owner': 0,
    'Second Owner': 1,
    'Third Owner': 2,
    'Fourth & Above Owner': 3,
    'Test Drive Car': 0
})


/tmp/ipython-input-2625902302.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['owner'] = df['owner'].replace({


In [ ]:
# Extract car model (first two words usually represent the model)
df['car_model'] = df['name'].apply(lambda x: " ".join(x.split()[:2]))


In [ ]:
df_model = df[['year', 'km_driven', 'fuel', 'seller_type',
               'transmission', 'owner', 'seats', 'engine', 'max_power',
               'car_model', 'selling_price']]



df_model = df_model.dropna()


In [ ]:
X = df_model.drop('selling_price', axis=1)
y = df_model['selling_price']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [ ]:
numeric_features = ['year', 'km_driven', 'owner', 'seats', 'engine', 'max_power']
categorical_features = ['fuel', 'seller_type', 'transmission', 'car_model']


preprocess = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

model = RandomForestRegressor(n_estimators=250, random_state=42)

pipeline = Pipeline([
    ('preprocess', preprocess),
    ('model', model)
])


In [ ]:
X = df_model.drop("selling_price", axis=1)
y = df_model["selling_price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['year', 'km_driven', 'owner',
                                                   'seats', 'engine',
                                                   'max_power']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['fuel', 'seller_type',
                                                   'transmission',
                                                   'car_model'])])),
                ('model',
                 RandomForestRegressor(n_estimators=250, random_state=42))])

In [ ]:

y_pred = pipeline.predict(X_test)

print("R2 Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


R2 Score: 0.9844575963600578
MAE: 59902.258461670375


In [ ]:
joblib.dump(pipeline, "car_price_model.pkl")
from google.colab import files
files.download("car_price_model.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>